In [ ]:
from lightfm import LightFM
from lightfm.data import Dataset
import pandas as pd

# Prepare dataset
df = pd.read_csv("temp.csv")
dataset = Dataset()
dataset.fit(df['user'], df['name'])

(interactions, weights) = dataset.build_interactions([(row['user'], row['name'], row['rating']) for _, row in df.iterrows()])

# Train model
model = LightFM(no_components=10, loss='bpr')  # 'warp' good for implicit, use 'mse' or 'logistic' for explicit
model.fit(interactions, epochs=30, num_threads=4)

/opt/homebrew/lib/python3.11/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [4]:
from lightfm.evaluation import precision_at_k

print("Precision:", precision_at_k(model, interactions, k=10).mean())


Precision: 0.99840003


In [ ]:
import numpy as np

user_id = 0  # internal LightFM ID
n_items = interactions.shape[1]

scores = model.predict(user_id, np.arange(n_items))
top_items = np.argsort(-scores)

# To map back to original names:
user_id_map, item_id_map, _, _ = dataset.mapping()
reverse_item_map = {v: k for k, v in item_id_map.items()}
recommended_items = [reverse_item_map[i] for i in top_items[:5]]
print("Top recommendations:", recommended_items)


Top recommendations: ['user83', 'user60', 'user95', 'user72', 'user66']
